# Import the necessary libraries

In [1]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import numpy as np  # Make sure numpy is imported
import tensorflow as tf

# Configure logging
logging.basicConfig(filename='../logs/predict_store_sales_with_deep_learning.log',
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
  
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from load_data import Load_Data
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import mean_squared_error, mean_absolute_error


# Load data set

In [2]:
logging.info("Load cleaned store data")

# Create an instance of CSVReader
df = Load_Data('../data/combined_cleaned_data.csv')

# Load the data
df.load_data()

# Get the loaded data
df = df.get_data()


Data successfully loaded from ../data/combined_cleaned_data.csv


In [3]:
print(df.dtypes)


Store                          int64
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
DayOfWeek                      int64
Date                          object
Sales                        float64
Customers                      int64
Open                           int64
Promo                          int64
SchoolHoliday                  int64
StoreType_b                     bool
StoreType_c                     bool
StoreType_d                     bool
Assortment_b                    bool
Assortment_c                    bool
StateHoliday_a                  bool
StateHoliday_b                  bool
StateHoliday_c                  bool
IsWeekend                      int64
MonthDay                       int64
IsBeginningOfMonth             int64
IsMidMonth                     int64
I

# LSTM Implementation

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

# Load your DataFrame
# df = pd.read_csv('your_data.csv')  # Example of loading your data

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Prepare the data
# Prepare the data with all relevant features
data = df[['Sales', 'CompetitionDistance', 'Promo2', 'DayOfWeek', 'Store', 
            'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 
            'Promo2SinceWeek', 'Promo2SinceYear', 'IsWeekend', 
            'MonthDay', 'SchoolHoliday', 'StoreType_b', 
            'StoreType_c', 'StoreType_d', 'Assortment_b', 
            'Assortment_c', 'StateHoliday_a', 'StateHoliday_b', 
            'StateHoliday_c', 'IsBeginningOfMonth', 'IsMidMonth', 
            'IsEndOfMonth']]

# Create target variable with differencing
data['Sales'] = data['Sales'].diff()
data['target'] = data['Sales'].shift(-1)
data.dropna(inplace=True)  # Remove rows with NaN values

# One-hot encode the 'PromoInterval' column if necessary
data = pd.get_dummies(data, columns=['PromoInterval'], drop_first=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(data)

# Prepare the data for LSTM
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :-1])  # All columns except target
        y.append(data[i + time_step, -1])  # Target column
    return np.array(X), np.array(y)

time_step = 10  # Use the last 10 days to predict the next day
X, y = create_dataset(scaled_data, time_step)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape features for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions if necessary
predictions = scaler.inverse_transform(predictions)

# Evaluate your model
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Test RMSE: {rmse:.4f}')


C:\Users\wubeshet.abera\AppData\Local\Temp\ipykernel_2844\2317894657.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sales'] = data['Sales'].diff()
C:\Users\wubeshet.abera\AppData\Local\Temp\ipykernel_2844\2317894657.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['Sales'].shift(-1)
C:\Users\wubeshet.abera\AppData\Local\Temp\ipykernel_2844\2317894657.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the

KeyError: "None of [Index(['PromoInterval'], dtype='object')] are in the [columns]"